# COMMON DEFINE

In [1]:
import numpy as np
import pandas as pd
import csv
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# NOMALIZE(TRAIN DATA)

In [2]:
train=pd.read_csv('data/KDDTrain+.csv', names=['Duration', 'Protocol type', 'Service', 'Flag', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])
test=pd.read_csv('data/KDDTest+.csv', names=['Duration', 'Protocol type', 'Service', 'Flag', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])

scaler = StandardScaler()
cols_train = train.select_dtypes(include=['float64','int64']).columns
cols_test = test.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(train.select_dtypes(include=['float64','int64']))
sc_test = scaler.transform(test.select_dtypes(include=['float64','int64']))
sc_traindf = pd.DataFrame(sc_train, columns = cols_train)
sc_testdf = pd.DataFrame(sc_test, columns = cols_test)
sc_traindf = sc_traindf.astype(np.float16)
sc_testdf = sc_testdf.astype(np.float16)
classtrain=train.select_dtypes(include=['object']).copy()
classtest=test.select_dtypes(include=['object']).copy()

train_y=classtrain['Class label']
train_y=pd.DataFrame(train_y, columns=['Class label'])
train=pd.concat([sc_traindf, train_y], axis=1)

test_y=classtest['Class label']
test_y=pd.DataFrame(test_y, columns=['Class label'])
test=pd.concat([sc_testdf, test_y], axis=1)

train.to_csv('data/nomalized_train.csv', sep=',', index=False)
test=pd.DataFrame(test, columns=train.columns)
test.to_csv('data/nomalized_test.csv', sep=',', na_rep='0', index=False)

# NOMALIZE(TEST DATA)

In [3]:
train=pd.read_csv('data/nomalized_train.csv')
test=pd.read_csv('data/nomalized_test.csv')
train_index=train['Class label'].value_counts().index
test_index=test['Class label'].value_counts().index
train_used_label=[]
train_unused_data=[]
for i in test_index:
    if i in train_index:
        train_used_label.append(i)

for i in range(0, len(test)):
    if test.iloc[i].values.tolist()[-1] not in train_used_label:
        train_unused_data.append(test.iloc[i].values.tolist())
        
train_unused_data=pd.DataFrame(train_unused_data, columns=['Duration', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])
train_unused_data.to_csv('data/test_unseen.csv', index=False)

# SAVE SUPPORT DATA(FOR ONE SHOT)

In [9]:
train=pd.read_csv('data/nomalized_train.csv')
test=pd.read_csv('data/nomalized_test.csv')

arr=[]
support_data=pd.DataFrame(columns=test.columns)
for i in range(0, len(train)):
    if train.loc[i]['Class label'] not in arr:
        arr.append(train.loc[i]['Class label'])

for i in range(0, len(test)):
    if test.loc[i]['Class label'] not in arr:
        arr.append(test.loc[i]['Class label'])
        support_data=support_data.append(test.loc[i])
support_data.to_csv('data/support_data/support_data_one_shot.csv', sep=',', index=False)

# DATA SPLIT BY CLASS LABEL(TRAIN)

In [ ]:
data=pd.read_csv('data/nomalized_train.csv', 'r', encoding='utf-8', delimiter=',')
class_label=data.select_dtypes(include=['object']).values
encoder=LabelEncoder()
encoder.fit(class_label)
class_label=encoder.transform(class_label)
data_by_label=[[] for i in range(0, len(data['Class label'].value_counts()))]

for i in range(0, len(data)):
    data_by_label[class_label[i]].append(data.iloc[i].values.tolist())

for i in data_by_label:
    i=pd.DataFrame(i, columns=['Duration', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])
    i.to_csv('data/data_by_label_train/'+i.iloc[0]['Class label']+'.csv', index=False)

# DATA SPLIT BY CLASS LABEL(TEST UNSEEN)

In [5]:
data=pd.read_csv('data/test_unseen.csv', 'r', encoding='utf-8', delimiter=',')
class_label=data.select_dtypes(include=['object']).values
encoder=LabelEncoder()
encoder.fit(class_label)
class_label=encoder.transform(class_label)
data_by_label=[[] for i in range(0, len(data['Class label'].value_counts()))]

for i in range(0, len(data)):
    data_by_label[class_label[i]].append(data.iloc[i].values.tolist())

for i in data_by_label:
    i=pd.DataFrame(i, columns=['Duration', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])
    i.to_csv('data/data_by_label_test_unseen/'+i.iloc[0]['Class label']+'.csv', index=False)

# OVERSAMPLING

In [3]:
file_list=os.listdir('data/data_by_label_train')
balanced_data=pd.DataFrame(columns=['Duration', 'Source bytes', 'Destination bytes', 'Land', 'Wrong fragment', 'Urgent', 'Hot', 'Number failed logins', 'Logged in', 'Num compromised', 'Root shell', 'Su attempted', 'Num root', 'Num file creations', 'Num shells', 'Num access files', 'Num outbound cmds', 'Is host login', 'Is guest login', 'Count', 'Srv count', 'Serror rate', 'Srv serror rate', 'Rerror rate', 'Srv rerror rate', 'Same srv rate', 'Diff srv rate', 'Srv diff host rate', 'Dst host count', 'Dst host srv count', 'Dst host same srv rate', 'Dst host diff srv rate', 'Dst host same src port rate', 'Dst host srv diff host rate', 'Dst host serror rate', 'Dst host srv serror rate', 'Dst host rerror rate', 'Dst host srv rerro rate', 'Class label'])
for i in file_list:
    data=pd.read_csv('data/data_by_label/'+i, 'r', encoding='utf-8', delimiter=',')
    while len(data)<60000:
        data=pd.concat([data, data], axis=0)
    data=data.iloc[0:60000]
    data.to_csv('data/balanced_data/'+i, index=False)
    balanced_data=balanced_data.append(data)
balanced_data=shuffle(balanced_data)
balanced_data.to_csv('data/balanced_data/ALL.csv', index=False)

# MAKE SIAMESE DATA(DIFF)

In [2]:
data=pd.read_csv('data/balanced_data/ALL.csv', 'r', encoding='utf-8', delimiter=',')
for i in range(0, 16):
    input1=data
    data=shuffle(data)
    input2=data
    siamesedatas=[]    
    for j in range(0, int(len(data))):
        if input1.iloc[j]['Class label']==input2.iloc[j]['Class label']:
            leftdata=input1.iloc[j].values[:-1]
            rightdata=input2.iloc[j].values[:-1]
            siamesedata=np.concatenate((leftdata, rightdata))
            siamesedata=np.concatenate((siamesedata, [0]))
            siamesedatas.append(siamesedata.tolist())
        else:
            leftdata=input1.iloc[j].values[:-1]
            rightdata=input2.iloc[j].values[:-1]
            siamesedata=np.concatenate((leftdata,rightdata))
            siamesedata=np.concatenate((siamesedata, [1]))
            siamesedatas.append(siamesedata.tolist())
    label=pd.DataFrame([1], index=['class'])
    siamesedatas=pd.DataFrame(siamesedatas, columns=input1.columns[:-1].append(input1.columns[:-1]).append(label.index))
    siamesedatas.to_csv('data/siamese_data_diff/'+str(i)+'.csv', sep=',', index=False)

# MAKE SIAMESE DATA(SAME)

In [5]:
file_list=os.listdir('data/balanced_data')
for i in range(0, 16):
    siamesedatas=[]
    for f in file_list:
        if f=='ALL.csv':
            continue
        
        data1=pd.read_csv('data/balanced_data/'+f, 'r', encoding='utf-8', delimiter=',')
        data2=shuffle(data1)
        data2.to_csv('data/balanced_data/'+f, sep=',', index=False)
        for j in range(0, int(len(data1))):
            leftdata=data1.iloc[j].values[:-1]
            rightdata=data2.iloc[j].values[:-1]
            siamesedata=np.concatenate((leftdata, rightdata))
            siamesedata=np.concatenate((siamesedata, [0]))
            siamesedatas.append(siamesedata.tolist())

    label=pd.DataFrame([1], index=['class'])
    siamesedatas=pd.DataFrame(siamesedatas, columns=data1.columns[:-1].append(data1.columns[:-1]).append(label.index))
    siamesedatas.to_csv('data/siamese_data_same/'+str(i)+'.csv', sep=',', index=False)

# MERGE

In [7]:
for i in range(0, 16):
    same=pd.read_csv('data/siamese_data_same/'+str(i)+'.csv', 'r', encoding='utf-8', delimiter=',')
    diff=pd.read_csv('data/siamese_data_diff/'+str(i)+'.csv', 'r', encoding='utf-8', delimiter=',')
    siamesedata_merge=pd.concat([same, diff], axis=0)
    siamesedata_merge=shuffle(siamesedata_merge)
    siamesedata_merge.to_csv('data/siamese_data_merge/'+str(i)+'.csv', sep=',', index=False)